# Download Scripts for Poster Images from Movielens 100k Dataset

Scripts are based on https://github.com/babu-thomas/movielens-posters

In [21]:
import csv
import urllib.parse
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

from tqdm import tqdm_notebook as tqdm, trange

#
# Scrap IMDB URLs
#
items = pd.read_csv('u.item.zip', compression='zip', sep='|', usecols=[0, 1],
                    encoding = "ISO-8859-1", names=['movie_id', 'movie_title'])
for _, row in tqdm(items.iterrows()): 
    domain = 'http://www.imdb.com'
    search_url = domain + '/find?q=' + urllib.parse.quote_plus(row.movie_title)
    with urllib.request.urlopen(search_url) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')

        try:
            title = soup.find('table', class_='findList').tr.a['href']
            movie_url = domain + title
            with open('movie_url.csv', 'a', newline='') as out_csv:
                writer = csv.writer(out_csv, delimiter=',')
                writer.writerow([row.movie_id, movie_url])
        except AttributeError:
            pass

In [4]:
#
# Scrap poster images and URLs
#
row_names = ['movie_id', 'movie_url']
urls = pd.read_csv('movie_url.csv', names=['movie_id', 'movie_url'])
for _, row in tqdm(urls.iterrows()):
        domain = 'http://www.imdb.com'
        with urllib.request.urlopen(row.movie_url) as response:
            html = response.read()
            soup = BeautifulSoup(html, 'html.parser')

            try:
                image_url = soup.find('div', class_='poster').a.img['src']
                extension = '.jpg'
                image_url = ''.join(image_url.partition('_')[0]) + extension
                filename = 'imdb-posters/' + row.movie_id + extension
                with urllib.request.urlopen(image_url) as response:
                    with open(filename, 'wb') as out_image:
                        out_image.write(response.read())
                    with open('movie_poster.csv', 'a', newline='') as out_csv:
                        writer = csv.writer(out_csv, delimiter=',')
                        writer.writerow([row.movie_id, row.image_url])
            except AttributeError:
                pass